In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings

warnings.filterwarnings('ignore')
np.random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [60]:
def xgboost_lr(File):
    
    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.8)
    X_train = train_df.drop(train_df.columns[0], axis=1)
    y_train = train_df[train_df.columns[0]]
    X_test = test_df.drop(test_df.columns[0], axis=1)
    y_test = test_df[test_df.columns[0]]
    
    
    dtrain = xgb.DMatrix(X_train.values,y_train.values)
    dtest = xgb.DMatrix(X_test.values, y_test.values)
    param = {'silent': 1, 'objective':'binary:logistic','max_depth': 7}
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    param['num_round']=30

    
    
    '''
    #------------------------------------------------------------------------------------------------------------
    #testing gridsearch
    clf=XGBClassifier(objective='binary:logistic',early_stopping_rounds=10)
    tuned_parameter = [{'n_estimators':[20,30,40], 'max_depth':[5,6,7]}]
    gs_clf = GridSearchCV(clf, tuned_parameter, cv=5, scoring='roc_auc')
    gs_clf.fit(X_train, y_train)
    param['num_round']=gs_clf.best_params_['n_estimators']
    param['max_depth']=gs_clf.best_params_['max_depth']
    print(param)
   
        
    #--------------------------------------------------------------------------------------------------------------   
    '''
    
    bst= xgb.train(param, dtrain)


    
    y_pred = bst.predict(dtrain, pred_leaf=True)
    num_leaf= 1000
    X_train_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_train_leaves[i][temp] += 1

    y_pred= bst.predict(dtest, pred_leaf=True)    
    X_test_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_test_leaves[i][temp] += 1
        
        
    bestC=0
    auc_best=0
    
    #---------------------------------------------------------------------------------
    # LR without regularization
    '''
    lm = LogisticRegression(penalty='l2',C=0.1) # logestic model construction
    lm.fit(X_train_leaves,y_train)  # fitting the data

    y_pred_est = lm.predict_proba(X_test_leaves)   # Give the probabilty on each label
    '''
    #---------------------------------------------------------------------------------------------
    #regularization applied testing
    c = np.array([1,0.5,0.1,0.05,0.01,0.005,0.001])
    for t in range(0,len(c)):
        lm = LogisticRegression(penalty='l2',C=c[t]) # logestic model construction
        lm.fit(X_train_leaves,y_train)  # fitting the data

        y_pred_est = lm.predict_proba(X_test_leaves)   # Give the probabilty on each label

        auc=roc_auc_score(y_test, y_pred_est[:,1])
        if auc_best < auc:
            auc_best=auc
            bestC=c[t]

    #---------------------------------------------------------------------------------------------
    
    
    print('best C value: %.2f' % bestC)
    print('XGBDT+LR auc 1: %.5f' % auc_best)
    
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_train_leaves, X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_test_leaves, X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('XGBDT+LR auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('XGBDT+LR time taken: %.2f'% f_time)
    
    #+NB

    
    start = time.clock()
    gnb= GaussianNB()

    gnb.fit(X_train_leaves, y_train)
    Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
    gnb_auc = roc_auc_score(y_test,Y_pred_nb)
    print('GBDT + GNB auc: %.5f'% gnb_auc)    
    '''    
    gnb.fit(X_train_ext, y_train)
    y_pred_gnb2=gnb.predict_proba(X_test_ext)[:,1]
    gnb_auc2=roc_auc_score(y_test,y_pred_gnb2)
    print('GNB auc2: %.5f' % gnb_auc2)
    '''
    '''#svc
    svc=SVC(probability=True)
    svc.fit(X_train_leaves, y_train)
    Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
    svc_auc=roc_auc_score(y_test,Y_pred_svc)
    print('GBDT + SVC auc: %.5f' % svc_auc)
    
    svc.fit(X_train_ext, y_train)
    y_pred_svc2=svc.predict_proba(X_test_ext)[:, 1]
    svc_auc2=roc_auc_score(y_test,y_pred_svc2)
    print('GBDT + SVC auc2: %.5f' % svc_auc2)'''
    
    '''#KNN
    knn=KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_train_leaves, y_train)
    Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
    knn_auc=roc_auc_score(y_test,Y_pred_knn)
    print('GBDT + KNN auc : %.5f' % knn_auc)
    
    knn.fit(X_train_ext, y_train)
    y_pred_knn2=knn.predict_proba(X_test_ext)[:, 1]
    knn_auc2=roc_auc_score(y_test,y_pred_knn2)
    print('GBDT + KNN auc2: %.5f' % knn_auc2)
    '''
    #perceptron

    perceptron = Perceptron()
    perceptron.fit(X_train_leaves, y_train)
    y_pred_perc=perceptron.predict(X_test_leaves)
    perc_auc=roc_auc_score(y_test,y_pred_perc)
    print('GBDT + Perceptron auc : %.5f' % perc_auc)
    
    perceptron.fit(X_train_ext, y_train)
    y_pred_perc2=perceptron.predict(X_test_ext)
    perc_auc2=roc_auc_score(y_test,y_pred_perc2)
    print('GBDT + Perceptron auc2 : %.5f' % perc_auc2 )
    
    #linear svc

    lin = LinearSVC()
    lin.fit(X_train_leaves, y_train)
    y_pred_lin=lin.predict(X_test_leaves)
    lin_auc=roc_auc_score(y_test,y_pred_lin)
    print('GBDT + Linear SVC auc : %.5f' % lin_auc)
    
    lin.fit(X_train_ext, y_train)
    y_pred_lin2=lin.predict(X_test_ext)
    lin_auc2=roc_auc_score(y_test,y_pred_lin2)
    print('GBDT + Linear SVC auc2 : %.5f' % lin_auc2)
    
    #SGD

    sgd = SGDClassifier(loss='log')
    sgd.fit(X_train_leaves, y_train)
    Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
    sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
    print('GBDT + SGD auc : %.5f' % sgd_auc)
    
    sgd.fit(X_train_ext, y_train)
    Y_pred_sgd2=sgd.predict_proba(X_test_ext)[:, 1]
    sgd_auc2=roc_auc_score(y_test,Y_pred_sgd2)
    print('GBDT + SGD auc2 : %.5f' % sgd_auc2)
    f_time =time.clock()-start
    print('GBDT + other classfier time taken: %.2f'% f_time)    
    
    #XGB
    start = time.clock()
    xgbc=XGBClassifier()
    xgbc.fit(X_train,y_train)
    Y_pred_xgb=xgbc.predict_proba(X_test)[:,1]
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('XGB auc : %.5f' % xgb_auc)
    
    '''#XGB with leaves

    xgbc.fit(X_train_leaves,y_train)
    Y_pred_xgb=xgbc.predict(X_test_leaves)
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('GBDT + XGB auc: %.5f' % xgb_auc)
    
    #XGB with features ext
    
    xgbc.fit(X_train_ext, y_train)
    y_pred_xgb2=xgbc.predict(X_test_ext)
    xgb_auc2=roc_auc_score(y_test,y_pred_xgb2)
    print('GBDT + XGB auc2: %.5f' %xgb_auc2)
    
    f_time =time.clock()-start
    print('XGB time taken: %.2f'% f_time)'''
    
    #lightGBM
    start = time.clock()
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    
    
    # specify your configurations as a dict
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
    
    }

    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                    verbose_eval=False)



    y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    # eval
    lgb_auc=roc_auc_score(y_test,y_pred_lgb)
    print('lightGBM auc : %.5f' % lgb_auc)
    
    lgb_train=lgb.Dataset(X_train_leaves, y_train)
    lgb_eval=lgb.Dataset(X_test_leaves, y_test, reference=lgb_train)
    gbm = lgb.train(params,
                  lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                   verbose_eval=False)
    y_pred_lgb2 =gbm.predict(X_test_leaves, num_iteration=gbm.best_iteration)
    lgb_auc2=roc_auc_score(y_test, y_pred_lgb2)
    
    print('GBDT + lightGBM auc : %.5f' % lgb_auc2)
    
    lgb_train=lgb.Dataset(X_train_ext, y_train)
    lgb_eval=lgb.Dataset(X_test_ext, y_test, reference=lgb_train)
    gbm = lgb.train(params,
                  lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                   verbose_eval=False)
    y_pred_lgb3 =gbm.predict(X_test_ext, num_iteration=gbm.best_iteration)
    lgb_auc3=roc_auc_score(y_test, y_pred_lgb3)
    
    print('GBDT + lightGBM auc2 : %.5f' % lgb_auc3)
    
    
    f_time=time.clock()-start
    print('lightGBM time taken: %.2f'% f_time)

In [3]:
example = pd.read_csv('example.csv')

In [10]:
xgboost_lr(example) #Trial 1

XGBDT+LR auc 1: 0.94069
XGBDT+LR auc 2: 0.62187
XGBDT+LR time taken: 6.23
GBDT + GNB auc: 0.86978
GBDT + Perceptron auc : 0.79588
GBDT + Perceptron auc2 : 0.48035
GBDT + Linear SVC auc : 0.85587
GBDT + Linear SVC auc2 : 0.46910
GBDT + SGD auc : 0.92947
GBDT + SGD auc2 : 0.52675
GBDT + other classfier time taken: 14.77
XGB auc : 0.93379
lightGBM auc : 0.93402
GBDT + lightGBM auc : 0.93368
GBDT + lightGBM auc2 : 0.93126
lightGBM time taken: 2.91


In [11]:
xgboost_lr(example) # Trial 2

XGBDT+LR auc 1: 0.92607
XGBDT+LR auc 2: 0.56891
XGBDT+LR time taken: 4.43
GBDT + GNB auc: 0.86347
GBDT + Perceptron auc : 0.85147
GBDT + Perceptron auc2 : 0.50000
GBDT + Linear SVC auc : 0.84385
GBDT + Linear SVC auc2 : 0.50617
GBDT + SGD auc : 0.91063
GBDT + SGD auc2 : 0.49945
GBDT + other classfier time taken: 13.48
XGB auc : 0.93151
lightGBM auc : 0.92760
GBDT + lightGBM auc : 0.92417
GBDT + lightGBM auc2 : 0.91973
lightGBM time taken: 3.01


In [12]:
xgboost_lr(example) # Trial 3

XGBDT+LR auc 1: 0.93185
XGBDT+LR auc 2: 0.54976
XGBDT+LR time taken: 4.46
GBDT + GNB auc: 0.85293
GBDT + Perceptron auc : 0.80626
GBDT + Perceptron auc2 : 0.51418
GBDT + Linear SVC auc : 0.82021
GBDT + Linear SVC auc2 : 0.50000
GBDT + SGD auc : 0.91307
GBDT + SGD auc2 : 0.50317
GBDT + other classfier time taken: 13.79
XGB auc : 0.92898
lightGBM auc : 0.92622
GBDT + lightGBM auc : 0.92848
GBDT + lightGBM auc2 : 0.93084
lightGBM time taken: 2.94


In [4]:
example=pd.read_csv('example2.csv')

In [15]:
xgboost_lr(example)  # trying another dataset 1

XGBDT+LR auc 1: 0.96331
XGBDT+LR auc 2: 0.63507
XGBDT+LR time taken: 8.96
GBDT + GNB auc: 0.89861
GBDT + Perceptron auc : 0.89369
GBDT + Perceptron auc2 : 0.72655
GBDT + Linear SVC auc : 0.87843
GBDT + Linear SVC auc2 : 0.78315
GBDT + SGD auc : 0.94790
GBDT + SGD auc2 : 0.72655
GBDT + other classfier time taken: 20.00
XGB auc : 0.95787
lightGBM auc : 0.94850
GBDT + lightGBM auc : 0.95372
GBDT + lightGBM auc2 : 0.95535
lightGBM time taken: 3.39


In [16]:
xgboost_lr(example)  # trying another dataset 2

XGBDT+LR auc 1: 0.96576
XGBDT+LR auc 2: 0.60730
XGBDT+LR time taken: 6.12
GBDT + GNB auc: 0.91934
GBDT + Perceptron auc : 0.89126
GBDT + Perceptron auc2 : 0.73162
GBDT + Linear SVC auc : 0.88857
GBDT + Linear SVC auc2 : 0.80514
GBDT + SGD auc : 0.95926
GBDT + SGD auc2 : 0.59079
GBDT + other classfier time taken: 19.95
XGB auc : 0.96142
lightGBM auc : 0.95631
GBDT + lightGBM auc : 0.95756
GBDT + lightGBM auc2 : 0.95847
lightGBM time taken: 3.52


In [17]:
xgboost_lr(example)  # trying another dataset 3

XGBDT+LR auc 1: 0.96155
XGBDT+LR auc 2: 0.64812
XGBDT+LR time taken: 6.17
GBDT + GNB auc: 0.90149
GBDT + Perceptron auc : 0.86405
GBDT + Perceptron auc2 : 0.51925
GBDT + Linear SVC auc : 0.86805
GBDT + Linear SVC auc2 : 0.57949
GBDT + SGD auc : 0.94537
GBDT + SGD auc2 : 0.72383
GBDT + other classfier time taken: 18.55
XGB auc : 0.95654
lightGBM auc : 0.95294
GBDT + lightGBM auc : 0.95554
GBDT + lightGBM auc2 : 0.95780
lightGBM time taken: 3.54


In [65]:
xgboost_lr(example) #testing params1

XGBDT+LR auc 1: 0.96367
XGBDT+LR auc 2: 0.61418
XGBDT+LR time taken: 12.20
GBDT + GNB auc: 0.89201
GBDT + Perceptron auc : 0.87319
GBDT + Perceptron auc2 : 0.62389
GBDT + Linear SVC auc : 0.87117
GBDT + Linear SVC auc2 : 0.79208
GBDT + SGD auc : 0.95278
GBDT + SGD auc2 : 0.72595
GBDT + other classfier time taken: 42.19
XGB auc : 0.95397
lightGBM auc : 0.95063
GBDT + lightGBM auc : 0.95417
GBDT + lightGBM auc2 : 0.95519
lightGBM time taken: 5.89


In [66]:
xgboost_lr(example) #testing params2

XGBDT+LR auc 1: 0.96145
XGBDT+LR auc 2: 0.71624
XGBDT+LR time taken: 11.25
GBDT + GNB auc: 0.88621
GBDT + Perceptron auc : 0.87716
GBDT + Perceptron auc2 : 0.72668
GBDT + Linear SVC auc : 0.88686
GBDT + Linear SVC auc2 : 0.73375
GBDT + SGD auc : 0.95541
GBDT + SGD auc2 : 0.72161
GBDT + other classfier time taken: 42.13
XGB auc : 0.95593
lightGBM auc : 0.95389
GBDT + lightGBM auc : 0.95290
GBDT + lightGBM auc2 : 0.95576
lightGBM time taken: 5.74


In [67]:
xgboost_lr(example) #testing params3

XGBDT+LR auc 1: 0.95582
XGBDT+LR auc 2: 0.62365
XGBDT+LR time taken: 11.09
GBDT + GNB auc: 0.89040
GBDT + Perceptron auc : 0.85024
GBDT + Perceptron auc2 : 0.73065
GBDT + Linear SVC auc : 0.87626
GBDT + Linear SVC auc2 : 0.57119
GBDT + SGD auc : 0.94661
GBDT + SGD auc2 : 0.73191
GBDT + other classfier time taken: 41.31
XGB auc : 0.94716
lightGBM auc : 0.94567
GBDT + lightGBM auc : 0.94859
GBDT + lightGBM auc2 : 0.94695
lightGBM time taken: 6.25


In [81]:
xgboost_lr(example) #testing grid search num_round : 30~50, max_depth : 5~8

{'silent': 1, 'objective': 'binary:logistic', 'num_round': 50, 'max_depth': 7}
XGBDT+LR auc 1: 0.96095
XGBDT+LR auc 2: 0.62529
XGBDT+LR time taken: 80.81
GBDT + GNB auc: 0.91951
GBDT + Perceptron auc : 0.86843
GBDT + Perceptron auc2 : 0.50000
GBDT + Linear SVC auc : 0.88063
GBDT + Linear SVC auc2 : 0.78430
GBDT + SGD auc : 0.95316
GBDT + SGD auc2 : 0.73162
GBDT + other classfier time taken: 5.90
XGB auc : 0.95803
lightGBM auc : 0.95525
GBDT + lightGBM auc : 0.95336
GBDT + lightGBM auc2 : 0.95493
lightGBM time taken: 1.65


In [83]:
xgboost_lr(example) #testing grid search  num_round : 50 ~ 70, max_depth : 7~10

{'silent': 1, 'objective': 'binary:logistic', 'num_round': 70, 'max_depth': 9}
XGBDT+LR auc 1: 0.95251
XGBDT+LR auc 2: 0.63956
XGBDT+LR time taken: 133.74
GBDT + GNB auc: 0.88624
GBDT + Perceptron auc : 0.86681
GBDT + Perceptron auc2 : 0.72665
GBDT + Linear SVC auc : 0.85990
GBDT + Linear SVC auc2 : 0.55178
GBDT + SGD auc : 0.94239
GBDT + SGD auc2 : 0.72288
GBDT + other classfier time taken: 6.08
XGB auc : 0.94968
lightGBM auc : 0.94232
GBDT + lightGBM auc : 0.93535
GBDT + lightGBM auc2 : 0.94258
lightGBM time taken: 1.55


In [84]:
xgboost_lr(example) #testing grid search with early stopping

{'silent': 1, 'objective': 'binary:logistic', 'num_round': 70, 'max_depth': 8}
XGBDT+LR auc 1: 0.95577
XGBDT+LR auc 2: 0.61168
XGBDT+LR time taken: 131.92
GBDT + GNB auc: 0.90123
GBDT + Perceptron auc : 0.86758
GBDT + Perceptron auc2 : 0.50000
GBDT + Linear SVC auc : 0.86351
GBDT + Linear SVC auc2 : 0.76937
GBDT + SGD auc : 0.94711
GBDT + SGD auc2 : 0.71315
GBDT + other classfier time taken: 6.03
XGB auc : 0.94839
lightGBM auc : 0.94590
GBDT + lightGBM auc : 0.94741
GBDT + lightGBM auc2 : 0.94994
lightGBM time taken: 1.65


In [7]:
xgboost_lr(example) #testing grid search  num_round : 20~40, max_depth : 5~7

{'silent': 1, 'objective': 'binary:logistic', 'num_round': 40, 'max_depth': 7}
XGBDT+LR auc 1: 0.95607
XGBDT+LR auc 2: 0.64764
XGBDT+LR time taken: 36.30
GBDT + GNB auc: 0.90986
GBDT + Perceptron auc : 0.86768
GBDT + Perceptron auc2 : 0.50000
GBDT + Linear SVC auc : 0.87193
GBDT + Linear SVC auc2 : 0.78617
GBDT + SGD auc : 0.94976
GBDT + SGD auc2 : 0.74507
GBDT + other classfier time taken: 1.42
XGB auc : 0.95050
lightGBM auc : 0.94357
GBDT + lightGBM auc : 0.95039
GBDT + lightGBM auc2 : 0.95138
lightGBM time taken: 0.96


In [10]:
xgboost_lr(example) #testing grid search  num_round : 20~40, max_depth : 5~7

{'silent': 1, 'objective': 'binary:logistic', 'num_round': 40, 'max_depth': 7}
XGBDT+LR auc 1: 0.95121
XGBDT+LR auc 2: 0.64968
XGBDT+LR time taken: 38.45
GBDT + GNB auc: 0.89994
GBDT + Perceptron auc : 0.83240
GBDT + Perceptron auc2 : 0.73587
GBDT + Linear SVC auc : 0.86004
GBDT + Linear SVC auc2 : 0.78863
GBDT + SGD auc : 0.93846
GBDT + SGD auc2 : 0.51573
GBDT + other classfier time taken: 5.99
XGB auc : 0.95007
lightGBM auc : 0.94953
GBDT + lightGBM auc : 0.94652
GBDT + lightGBM auc2 : 0.94854
lightGBM time taken: 1.68


In [16]:
xgboost_lr(example) 

XGBDT+LR auc 1: 0.95769
XGBDT+LR auc 2: 0.58779
XGBDT+LR time taken: 1.99
GBDT + GNB auc: 0.90880
GBDT + Perceptron auc : 0.85485
GBDT + Perceptron auc2 : 0.72575
GBDT + Linear SVC auc : 0.86000
GBDT + Linear SVC auc2 : 0.70816
GBDT + SGD auc : 0.94906
GBDT + SGD auc2 : 0.50185
GBDT + other classfier time taken: 5.97
XGB auc : 0.95242
lightGBM auc : 0.94748
GBDT + lightGBM auc : 0.95043
GBDT + lightGBM auc2 : 0.95230
lightGBM time taken: 1.47


In [17]:
xgboost_lr(example) 

XGBDT+LR auc 1: 0.96265
XGBDT+LR auc 2: 0.60898
XGBDT+LR time taken: 1.99
GBDT + GNB auc: 0.91668
GBDT + Perceptron auc : 0.84197
GBDT + Perceptron auc2 : 0.72576
GBDT + Linear SVC auc : 0.89128
GBDT + Linear SVC auc2 : 0.57192
GBDT + SGD auc : 0.95019
GBDT + SGD auc2 : 0.72863
GBDT + other classfier time taken: 6.01
XGB auc : 0.95659
lightGBM auc : 0.95572
GBDT + lightGBM auc : 0.95514
GBDT + lightGBM auc2 : 0.95774
lightGBM time taken: 1.61


In [18]:
xgboost_lr(example) 

XGBDT+LR auc 1: 0.95512
XGBDT+LR auc 2: 0.60247
XGBDT+LR time taken: 2.09
GBDT + GNB auc: 0.92289
GBDT + Perceptron auc : 0.85925
GBDT + Perceptron auc2 : 0.74393
GBDT + Linear SVC auc : 0.87706
GBDT + Linear SVC auc2 : 0.78987
GBDT + SGD auc : 0.94682
GBDT + SGD auc2 : 0.75873
GBDT + other classfier time taken: 6.54
XGB auc : 0.95377
lightGBM auc : 0.95109
GBDT + lightGBM auc : 0.94884
GBDT + lightGBM auc2 : 0.95355
lightGBM time taken: 1.65


In [19]:
xgboost_lr(example) 

XGBDT+LR auc 1: 0.96387
XGBDT+LR auc 2: 0.61440
XGBDT+LR time taken: 1.99
GBDT + GNB auc: 0.92455
GBDT + Perceptron auc : 0.88326
GBDT + Perceptron auc2 : 0.33190
GBDT + Linear SVC auc : 0.88753
GBDT + Linear SVC auc2 : 0.63559
GBDT + SGD auc : 0.94128
GBDT + SGD auc2 : 0.70941
GBDT + other classfier time taken: 6.21
XGB auc : 0.95834
lightGBM auc : 0.95581
GBDT + lightGBM auc : 0.95743
GBDT + lightGBM auc2 : 0.95945
lightGBM time taken: 1.46


In [25]:
xgboost_lr(example)  # eta=0.1 applied

XGBDT+LR auc 1: 0.95235
XGBDT+LR auc 2: 0.65865
XGBDT+LR time taken: 2.12
GBDT + GNB auc: 0.90088
GBDT + Perceptron auc : 0.86562
GBDT + Perceptron auc2 : 0.71858
GBDT + Linear SVC auc : 0.87337
GBDT + Linear SVC auc2 : 0.49802
GBDT + SGD auc : 0.94300
GBDT + SGD auc2 : 0.50000
GBDT + other classfier time taken: 6.14
XGB auc : 0.95269
lightGBM auc : 0.94858
GBDT + lightGBM auc : 0.94887
GBDT + lightGBM auc2 : 0.95380
lightGBM time taken: 1.62


In [26]:
xgboost_lr(example)  # eta=0.1 applied

XGBDT+LR auc 1: 0.95633
XGBDT+LR auc 2: 0.62859
XGBDT+LR time taken: 1.94
GBDT + GNB auc: 0.93053
GBDT + Perceptron auc : 0.86511
GBDT + Perceptron auc2 : 0.50244
GBDT + Linear SVC auc : 0.88180
GBDT + Linear SVC auc2 : 0.74764
GBDT + SGD auc : 0.94139
GBDT + SGD auc2 : 0.74994
GBDT + other classfier time taken: 6.40
XGB auc : 0.95804
lightGBM auc : 0.95482
GBDT + lightGBM auc : 0.94831
GBDT + lightGBM auc2 : 0.95534
lightGBM time taken: 1.65


In [27]:
xgboost_lr(example)  # eta=0.01 applied

XGBDT+LR auc 1: 0.95287
XGBDT+LR auc 2: 0.64091
XGBDT+LR time taken: 1.94
GBDT + GNB auc: 0.91910
GBDT + Perceptron auc : 0.84249
GBDT + Perceptron auc2 : 0.72257
GBDT + Linear SVC auc : 0.87561
GBDT + Linear SVC auc2 : 0.55202
GBDT + SGD auc : 0.94272
GBDT + SGD auc2 : 0.73307
GBDT + other classfier time taken: 6.09
XGB auc : 0.95367
lightGBM auc : 0.95260
GBDT + lightGBM auc : 0.94656
GBDT + lightGBM auc2 : 0.95233
lightGBM time taken: 1.63


In [28]:
xgboost_lr(example)  # eta=0.01 applied

XGBDT+LR auc 1: 0.94076
XGBDT+LR auc 2: 0.64622
XGBDT+LR time taken: 2.04
GBDT + GNB auc: 0.90656
GBDT + Perceptron auc : 0.85477
GBDT + Perceptron auc2 : 0.73244
GBDT + Linear SVC auc : 0.86629
GBDT + Linear SVC auc2 : 0.54074
GBDT + SGD auc : 0.92358
GBDT + SGD auc2 : 0.63984
GBDT + other classfier time taken: 6.13
XGB auc : 0.94855
lightGBM auc : 0.94386
GBDT + lightGBM auc : 0.93581
GBDT + lightGBM auc2 : 0.94244
lightGBM time taken: 1.49


In [30]:
xgboost_lr(example)  # eta=0.1 applied

XGBDT+LR auc 1: 0.94764
XGBDT+LR auc 2: 0.62906
XGBDT+LR time taken: 1.98
GBDT + GNB auc: 0.91031
GBDT + Perceptron auc : 0.85817
GBDT + Perceptron auc2 : 0.50126
GBDT + Linear SVC auc : 0.86864
GBDT + Linear SVC auc2 : 0.54670
GBDT + SGD auc : 0.93948
GBDT + SGD auc2 : 0.73156
GBDT + other classfier time taken: 6.09
XGB auc : 0.94981
lightGBM auc : 0.94661
GBDT + lightGBM auc : 0.93722
GBDT + lightGBM auc2 : 0.94639
lightGBM time taken: 1.49


In [31]:
xgboost_lr(example)  # eta=0.1 applied

XGBDT+LR auc 1: 0.94759
XGBDT+LR auc 2: 0.60711
XGBDT+LR time taken: 1.97
GBDT + GNB auc: 0.91084
GBDT + Perceptron auc : 0.86425
GBDT + Perceptron auc2 : 0.72784
GBDT + Linear SVC auc : 0.87225
GBDT + Linear SVC auc2 : 0.64848
GBDT + SGD auc : 0.93893
GBDT + SGD auc2 : 0.74313
GBDT + other classfier time taken: 6.39
XGB auc : 0.94943
lightGBM auc : 0.94146
GBDT + lightGBM auc : 0.94008
GBDT + lightGBM auc2 : 0.94731
lightGBM time taken: 1.50


In [32]:
xgboost_lr(example)  # eta=0.1 applied

XGBDT+LR auc 1: 0.95941
XGBDT+LR auc 2: 0.62627
XGBDT+LR time taken: 1.98
GBDT + GNB auc: 0.91689
GBDT + Perceptron auc : 0.83283
GBDT + Perceptron auc2 : 0.69812
GBDT + Linear SVC auc : 0.88228
GBDT + Linear SVC auc2 : 0.77100
GBDT + SGD auc : 0.95424
GBDT + SGD auc2 : 0.50953
GBDT + other classfier time taken: 6.26
XGB auc : 0.95780
lightGBM auc : 0.95386
GBDT + lightGBM auc : 0.95282
GBDT + lightGBM auc2 : 0.95575
lightGBM time taken: 1.58


In [33]:
xgboost_lr(example)  # eta=0.1 applied

XGBDT+LR auc 1: 0.94350
XGBDT+LR auc 2: 0.60106
XGBDT+LR time taken: 2.00
GBDT + GNB auc: 0.89834
GBDT + Perceptron auc : 0.85667
GBDT + Perceptron auc2 : 0.71141
GBDT + Linear SVC auc : 0.86052
GBDT + Linear SVC auc2 : 0.76073
GBDT + SGD auc : 0.92719
GBDT + SGD auc2 : 0.43370
GBDT + other classfier time taken: 6.17
XGB auc : 0.94340
lightGBM auc : 0.93661
GBDT + lightGBM auc : 0.93736
GBDT + lightGBM auc2 : 0.94299
lightGBM time taken: 1.74


In [35]:
xgboost_lr(example) # without eta

XGBDT+LR auc 1: 0.95669
XGBDT+LR auc 2: 0.61759
XGBDT+LR time taken: 2.02
GBDT + GNB auc: 0.91129
GBDT + Perceptron auc : 0.82474
GBDT + Perceptron auc2 : 0.74395
GBDT + Linear SVC auc : 0.87207
GBDT + Linear SVC auc2 : 0.55048
GBDT + SGD auc : 0.94817
GBDT + SGD auc2 : 0.50000
GBDT + other classfier time taken: 5.98
XGB auc : 0.95212
lightGBM auc : 0.94959
GBDT + lightGBM auc : 0.95181
GBDT + lightGBM auc2 : 0.95142
lightGBM time taken: 1.65


In [61]:
xgboost_lr(example)  # regularizing lr

best C value: 0.10
XGBDT+LR auc 1: 0.95269
XGBDT+LR auc 2: 0.65136
XGBDT+LR time taken: 5.71
GBDT + GNB auc: 0.90510
GBDT + Perceptron auc : 0.83223
GBDT + Perceptron auc2 : 0.73890
GBDT + Linear SVC auc : 0.87233
GBDT + Linear SVC auc2 : 0.59625
GBDT + SGD auc : 0.94749
GBDT + SGD auc2 : 0.73563
GBDT + other classfier time taken: 6.09
XGB auc : 0.94689
lightGBM auc : 0.94150
GBDT + lightGBM auc : 0.94346
GBDT + lightGBM auc2 : 0.94659
lightGBM time taken: 1.59


In [62]:
xgboost_lr(example)  # regularizing lr

best C value: 0.10
XGBDT+LR auc 1: 0.95645
XGBDT+LR auc 2: 0.86918
XGBDT+LR time taken: 5.62
GBDT + GNB auc: 0.90405
GBDT + Perceptron auc : 0.85365
GBDT + Perceptron auc2 : 0.53253
GBDT + Linear SVC auc : 0.87563
GBDT + Linear SVC auc2 : 0.56682
GBDT + SGD auc : 0.94613
GBDT + SGD auc2 : 0.35602
GBDT + other classfier time taken: 6.28
XGB auc : 0.94900
lightGBM auc : 0.95176
GBDT + lightGBM auc : 0.94859
GBDT + lightGBM auc2 : 0.95330
lightGBM time taken: 1.69


In [68]:
xgboost_lr(example)  # regularizing lr

best C value: 0.50
XGBDT+LR auc 1: 0.95646
XGBDT+LR auc 2: 0.65687
XGBDT+LR time taken: 5.50
GBDT + GNB auc: 0.91188
GBDT + Perceptron auc : 0.86114
GBDT + Perceptron auc2 : 0.72662
GBDT + Linear SVC auc : 0.87547
GBDT + Linear SVC auc2 : 0.78412
GBDT + SGD auc : 0.95358
GBDT + SGD auc2 : 0.72579
GBDT + other classfier time taken: 6.18
XGB auc : 0.95581
lightGBM auc : 0.95370
GBDT + lightGBM auc : 0.94876
GBDT + lightGBM auc2 : 0.95032
lightGBM time taken: 1.62
